In [1]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion

In [2]:
!pip install tqdm

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic


In [3]:
TRAIN_DIR = 'train_n'
GOOD_DIR = 'good_ash'
BAD_DIR = 'bad_ash'
TEST_DIR = 'final_test'
IMG_SIZE = 224
LR = 1e-3

In [4]:
MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic') # just so we remember which saved model is which, sizes must match


In [5]:
def create_train_data():
    training_data = []
    
    for img in tqdm(os.listdir(GOOD_DIR)):
        label = [1.0,0.0] # 'good'
        path = os.path.join(GOOD_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    
    for img in tqdm(os.listdir(BAD_DIR)):
        label = [0.0,1.0] #'bad'
        path = os.path.join(BAD_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    
    return training_data

In [6]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)

In [7]:
train_data = create_train_data()
#test_data = process_test_data()
# If you have already created the dataset:
#train_data = np.load('train_data.npy')

100%|██████████| 322/322 [00:05<00:00, 55.60it/s]


In [12]:
!pip install tflearn

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic


In [13]:
 !pip install tensorflow_cpu


Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic


In [14]:
!pip install --upgrade tflearn 

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Requirement already up-to-date: tflearn in /home/abed/jupyter_py3/lib/python3.6/site-packages (0.3.2)


In [15]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [16]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

In [17]:
train = train_data[:-10]
test = train_data[-10:]

In [18]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

In [19]:
model.fit({'input': X}, {'targets': Y}, n_epoch=1, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=10, show_metric=True, run_id=MODEL_NAME)

Training Step: 9  | total loss: 11.58082 | time: 29.503s
| Adam | epoch: 001 | loss: 11.58082 - acc: 0.4970 -- iter: 576/636
Training Step: 10  | total loss: 11.90665 | time: 34.845s
| Adam | epoch: 001 | loss: 11.90665 - acc: 0.4829 | val_loss: 9.21034 - val_acc: 0.6000 -- iter: 636/636
--


#the End of model 1

In [20]:
krl = 3
drp_out_per = 0.2
epoc_n = 10

In [21]:
import tensorflow as tf


tf.reset_default_graph()
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, krl, activation='relu')
convnet = max_pool_2d(convnet, krl)
#convnet = dropout(convnet, drp_out_per)

convnet = conv_2d(convnet, 64, krl, activation='relu')
convnet = max_pool_2d(convnet, krl)
#convnet = dropout(convnet, drp_out_per)

convnet = conv_2d(convnet, 128, krl, activation='relu')
convnet = max_pool_2d(convnet, krl)
#convnet = dropout(convnet, drp_out_per)

convnet = conv_2d(convnet, 256, krl, activation='relu') #32
convnet = max_pool_2d(convnet, krl)
#convnet = dropout(convnet, drp_out_per)

convnet = conv_2d(convnet, 512, krl, activation='relu') #64
convnet = max_pool_2d(convnet, krl)
#convnet = dropout(convnet, drp_out_per)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, drp_out_per)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')




In [ ]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

train = train_data[:-10]
test = train_data[-10:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

model.fit({'input': X}, {'targets': Y}, n_epoch=epoc_n, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

---------------------------------
Run id: dogsvscats-0.001-2conv-basic.model
Log directory: log/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 636
Validation samples: 10
--
